In [1]:
import pandas as pd
import boto3
import numpy as np

In [2]:
bucket = 'datascience-sagemaker-talitasantos-ac5'

In [3]:
dados_brutos_arquivo = 'Brasileirao_Matches.csv'

try:
    os.mkdir('dados')
except:
    pass

dados = pd.read_csv("s3://{}/{}".format(bucket, "Brasileirao_Matches.csv"))
pd.set_option('display.max_rows', 20) 
dados

,datetime,home_team,home_team_state,away_team,away_team_state,home_goal,away_goal,season,round
0,2012-05-19 18:30:00,Palmeiras,SP,Portuguesa,SP,1.0,1.0,2012,1
1,2012-05-19 18:30:00,Sport,PE,Flamengo,RJ,1.0,1.0,2012,1
2,2012-05-19 21:00:00,Figueirense,SC,Náutico,PE,2.0,1.0,2012,1
3,2012-05-20 16:00:00,Ponte Preta,SP,Atlético,MG,0.0,1.0,2012,1
4,2012-05-20 16:00:00,Corinthians,SP,Fluminense,RJ,0.0,1.0,2012,1
...,...,...,...,...,...,...,...,...,...
3795,2021-12-09 21:30:00,Fortaleza,CE,Bahia,BA,NaN,NaN,2021,38
3796,2021-12-09 21:30:00,Sport,PE,Athletico Paranaense,PR,NaN,NaN,2021,38
3797,2021-12-09 21:30:00,Red Bull Bragantino,SP,Internacional,RS,NaN,NaN,2021,38
3798,2021-12-09 21:30:00,Juventude,RS,Corinthians,SP,NaN,NaN,2021,38


In [4]:
dados = dados.drop(['home_team_state', 'away_team_state','round'], axis=1)

In [5]:
for x in dados:
  stage = 'pontos corridos'
  dados['stage'] = stage
dados.columns 

dados['home_goal']=dados['home_goal'].fillna('0')
dados['away_goal']=dados['away_goal'].fillna('0')

In [6]:
dados_brutos_arquivo = 'Libertadores_Matches.csv'

try:
    os.mkdir('dados1')
except:
    pass

dados1 = pd.read_csv("s3://{}/{}".format(bucket, "Libertadores_Matches.csv"))
pd.set_option('display.max_rows', 20) 
dados1

,datetime,home_team,away_team,home_goal,away_goal,season,stage
0,2013-02-12 20:15:00,Nacional (URU),Barcelona-EQU,2,2,2013.0,group stage
1,2013-02-13 22:00:00,Boca Juniors,Toluca,1,2,2013.0,group stage
2,2013-02-19 23:45:00,Toluca,Nacional (URU),2,3,2013.0,group stage
3,2013-02-27 19:45:00,Barcelona-EQU,Boca Juniors,1,2,2013.0,group stage
4,2013-03-07 00:15:00,Toluca,Barcelona-EQU,1,1,2013.0,group stage
...,...,...,...,...,...,...,...
1126,2021-09-22 21:30:00,Flamengo,Barcelona-EQU,2,0,2021.0,semifinals
1127,2021-09-29 21:30:00,Barcelona-EQU,Flamengo,0,2,2021.0,semifinals
1128,2021-09-21 21:30:00,Palmeiras,Atlético-MG,0,0,2021.0,semifinals
1129,2021-09-28 21:30:00,Atlético-MG,Palmeiras,1,1,2021.0,semifinals


In [7]:
dados1['home_goal'] = dados1['home_goal'].str.replace('-','0')
dados1['away_goal'] = dados1['away_goal'].str.replace('-','0')

In [8]:
dataset = pd.concat([dados, dados1])
dataset

,datetime,home_team,away_team,home_goal,away_goal,season,stage
0,2012-05-19 18:30:00,Palmeiras,Portuguesa,1,1,2012.0,pontos corridos
1,2012-05-19 18:30:00,Sport,Flamengo,1,1,2012.0,pontos corridos
2,2012-05-19 21:00:00,Figueirense,Náutico,2,1,2012.0,pontos corridos
3,2012-05-20 16:00:00,Ponte Preta,Atlético,0,1,2012.0,pontos corridos
4,2012-05-20 16:00:00,Corinthians,Fluminense,0,1,2012.0,pontos corridos
...,...,...,...,...,...,...,...
1126,2021-09-22 21:30:00,Flamengo,Barcelona-EQU,2,0,2021.0,semifinals
1127,2021-09-29 21:30:00,Barcelona-EQU,Flamengo,0,2,2021.0,semifinals
1128,2021-09-21 21:30:00,Palmeiras,Atlético-MG,0,0,2021.0,semifinals
1129,2021-09-28 21:30:00,Atlético-MG,Palmeiras,1,1,2021.0,semifinals


In [9]:
dataset['home_goal'] = dataset['home_goal'].astype('float')
dataset['away_goal'] = dataset['away_goal'].astype('float')

In [10]:
dataset['ALVO'] = dataset.apply(lambda x: 2 if x['home_goal'] > x['away_goal'] else (0 if x['home_goal'] < x['away_goal'] else 1), axis=1)

In [11]:
dataset

,datetime,home_team,away_team,home_goal,away_goal,season,stage,ALVO
0,2012-05-19 18:30:00,Palmeiras,Portuguesa,1.0,1.0,2012.0,pontos corridos,1
1,2012-05-19 18:30:00,Sport,Flamengo,1.0,1.0,2012.0,pontos corridos,1
2,2012-05-19 21:00:00,Figueirense,Náutico,2.0,1.0,2012.0,pontos corridos,2
3,2012-05-20 16:00:00,Ponte Preta,Atlético,0.0,1.0,2012.0,pontos corridos,0
4,2012-05-20 16:00:00,Corinthians,Fluminense,0.0,1.0,2012.0,pontos corridos,0
...,...,...,...,...,...,...,...,...
1126,2021-09-22 21:30:00,Flamengo,Barcelona-EQU,2.0,0.0,2021.0,semifinals,2
1127,2021-09-29 21:30:00,Barcelona-EQU,Flamengo,0.0,2.0,2021.0,semifinals,0
1128,2021-09-21 21:30:00,Palmeiras,Atlético-MG,0.0,0.0,2021.0,semifinals,1
1129,2021-09-28 21:30:00,Atlético-MG,Palmeiras,1.0,1.0,2021.0,semifinals,1


In [12]:
dataset['Pre_Home_Palmeiras'] = dataset['home_team'].apply(lambda x: 1 if x=='Palmeiras' else 0)
dataset['Pre_Home_Santos'] = dataset['home_team'].apply(lambda x: 1 if x=='Santos' else 0)
dataset['Pre_Home_SaoPaulo'] = dataset['home_team'].apply(lambda x: 1 if x=='São Paulo' else 0)
dataset['Pre_Home_Corinthians'] = dataset['home_team'].apply(lambda x: 1 if x=='Corinthians' else 0)
dataset['Pre_Home_Outros'] = dataset['home_team'].apply(
    lambda x: 1 if x!='Corinthians' and x!= 'São Paulo' and x!='Santos' and x!= 'Palmeiras' else 0)

dataset['Pre_Away_Palmeiras'] = dataset['away_team'].apply(lambda x: 1 if x=='Palmeiras' else 0)
dataset['Pre_Away_Santos'] = dataset['away_team'].apply(lambda x: 1 if x=='Santos' else 0)
dataset['Pre_Away_SaoPaulo'] = dataset['away_team'].apply(lambda x: 1 if x=='São Paulo' else 0)
dataset['Pre_Away_Corinthians'] = dataset['away_team'].apply(lambda x: 1 if x=='Corinthians' else 0)
dataset['Pre_Away_Outros'] = dataset['away_team'].apply(
    lambda x: 1 if x!='Corinthians' and x!= 'São Paulo' and x!='Santos' and x!= 'Palmeiras' else 0)

#normalização
dataset['Pre_home_goal'] = [1. if x>8 else (x-0)/(8-0) for x in dataset['home_goal']]
dataset['Pre_away_goal'] = [1. if x>7 else (x-0)/(7-0) for x in dataset['away_goal']]
dataset

,datetime,home_team,away_team,home_goal,away_goal,season,stage,ALVO,Pre_Home_Palmeiras,Pre_Home_Santos,Pre_Home_SaoPaulo,Pre_Home_Corinthians,Pre_Home_Outros,Pre_Away_Palmeiras,Pre_Away_Santos,Pre_Away_SaoPaulo,Pre_Away_Corinthians,Pre_Away_Outros,Pre_home_goal,Pre_away_goal
0,2012-05-19 18:30:00,Palmeiras,Portuguesa,1.0,1.0,2012.0,pontos corridos,1,1,0,0,0,0,0,0,0,0,1,0.125,0.142857
1,2012-05-19 18:30:00,Sport,Flamengo,1.0,1.0,2012.0,pontos corridos,1,0,0,0,0,1,0,0,0,0,1,0.125,0.142857
2,2012-05-19 21:00:00,Figueirense,Náutico,2.0,1.0,2012.0,pontos corridos,2,0,0,0,0,1,0,0,0,0,1,0.250,0.142857
3,2012-05-20 16:00:00,Ponte Preta,Atlético,0.0,1.0,2012.0,pontos corridos,0,0,0,0,0,1,0,0,0,0,1,0.000,0.142857
4,2012-05-20 16:00:00,Corinthians,Fluminense,0.0,1.0,2012.0,pontos corridos,0,0,0,0,1,0,0,0,0,0,1,0.000,0.142857
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1126,2021-09-22 21:30:00,Flamengo,Barcelona-EQU,2.0,0.0,2021.0,semifinals,2,0,0,0,0,1,0,0,0,0,1,0.250,0.000000
1127,2021-09-29 21:30:00,Barcelona-EQU,Flamengo,0.0,2.0,2021.0,semifinals,0,0,0,0,0,1,0,0,0,0,1,0.000,0.285714
1128,2021-09-21 21:30:00,Palmeiras,Atlético-MG,0.0,0.0,2021.0,semifinals,1,1,0,0,0,0,0,0,0,0,1,0.000,0.000000
1129,2021-09-28 21:30:00,Atlético-MG,Palmeiras,1.0,1.0,2021.0,semifinals,1,0,0,0,0,1,1,0,0,0,0,0.125,0.142857


In [13]:
dataset = dataset[dataset.home_team.isin(['Palmeiras', 'São Paulo', 'Santos', 'Corinthians']) | dataset.away_team.isin(['Palmeiras', 'São Paulo', 'Santos', 'Corinthians'])]

In [14]:
# atributos
X = dataset[['home_goal', 'away_goal',
    'Pre_Home_Palmeiras', 'Pre_Home_Santos',
    'Pre_Home_SaoPaulo', 'Pre_Home_Corinthians', 'Pre_Home_Outros',
    'Pre_Away_Palmeiras', 'Pre_Away_Santos', 'Pre_Away_SaoPaulo', 'Pre_Away_Corinthians',
    'Pre_home_goal', 'Pre_away_goal', 'Pre_Away_Outros']]

# categoria
y = dataset['ALVO']

In [15]:
treinamento, validacao, teste = np.split(dataset.sample(frac=1, random_state=88), [int(0.7 * len(dataset)), int(0.9 * len(dataset))]) 
len(dataset),len(treinamento), len(validacao), len(teste)

(1546, 1082, 309, 155)

In [16]:
treinamento.to_csv('treinamento.csv', index=False, header=False)
validacao.to_csv('validacao.csv', index=False, header=False)
teste.to_csv('teste.csv', index=False)

In [17]:
boto3.Session().resource('s3').Bucket(bucket).Object('treinamento.csv').upload_file('treinamento.csv')
boto3.Session().resource('s3').Bucket(bucket).Object('validacao.csv').upload_file('validacao.csv')
boto3.Session().resource('s3').Bucket(bucket).Object('teste.csv').upload_file('teste.csv')

In [18]:
import sagemaker
from sagemaker.session import TrainingInput
import numpy as np
from sagemaker.pytorch import PyTorch
import pandas as pd
from sklearn.metrics import classification_report

sagemaker_session = sagemaker.Session()

bucket = 'datascience-sagemaker-talitasantos-ac5'
prefix = 'mlp/campeonato'

instance_type = 'local'

role = sagemaker.get_execution_role()

In [19]:
#salvar arquivo csv
dataset.to_csv('data/train.csv')

# enviar para s3
sagemaker_session.upload_data(path='data', bucket=bucket, key_prefix=prefix)

's3://datascience-sagemaker-talitasantos-ac5/mlp/campeonato'

In [20]:
train_input = TrainingInput(
    "s3://{}/{}/{}".format(bucket, prefix, "train.csv"), content_type="csv"
)

In [21]:
estimator = PyTorch(entry_point='mlp.py',
                    base_name='campeonato_MLP',
                    role=role,
                    py_version='py3',
                    framework_version='1.8.0',
                    instance_count=1,
                    instance_type=instance_type,
                    hyperparameters={
                        'epochs': 100,
                    })

In [22]:
estimator.fit({"training": train_input})

Creating 0qtb57o8lf-algo-1-5z6nf ... 
Creating 0qtb57o8lf-algo-1-5z6nf ... done
Attaching to 0qtb57o8lf-algo-1-5z6nf
0qtb57o8lf-algo-1-5z6nf | 2022-05-31 23:19:18,832 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
0qtb57o8lf-algo-1-5z6nf | 2022-05-31 23:19:18,835 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
0qtb57o8lf-algo-1-5z6nf | 2022-05-31 23:19:18,847 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
0qtb57o8lf-algo-1-5z6nf | 2022-05-31 23:19:18,850 sagemaker_pytorch_container.training INFO     Invoking user training script.
0qtb57o8lf-algo-1-5z6nf | 2022-05-31 23:19:19,076 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
0qtb57o8lf-algo-1-5z6nf | 2022-05-31 23:19:19,091 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
0qtb57o8lf-algo-1-5z6nf | 2022-05-31 23:19:19,106 sagemaker-training-toolkit 

In [23]:
predictor = estimator.deploy(initial_instance_count=1, instance_type=instance_type, endpoint_name='datascience-campeonatos-mlp-6')

Attaching to daikjnsgyj-algo-1-uf0kq
daikjnsgyj-algo-1-uf0kq | ['torchserve', '--start', '--model-store', '/.sagemaker/ts/models', '--ts-config', '/etc/sagemaker-ts.properties', '--log-config', '/opt/conda/lib/python3.6/site-packages/sagemaker_pytorch_serving_container/etc/log4j.properties', '--models', 'model.mar']
daikjnsgyj-algo-1-uf0kq | 2022-05-31 23:19:45,216 [INFO ] main org.pytorch.serve.ModelServer - 
daikjnsgyj-algo-1-uf0kq | Torchserve version: 0.3.0
daikjnsgyj-algo-1-uf0kq | TS Home: /opt/conda/lib/python3.6/site-packages
daikjnsgyj-algo-1-uf0kq | Current directory: /
daikjnsgyj-algo-1-uf0kq | Temp directory: /home/model-server/tmp
daikjnsgyj-algo-1-uf0kq | Number of GPUs: 0
daikjnsgyj-algo-1-uf0kq | Number of CPUs: 2
daikjnsgyj-algo-1-uf0kq | Max heap size: 988 M
daikjnsgyj-algo-1-uf0kq | Python executable: /opt/conda/bin/python3.6
daikjnsgyj-algo-1-uf0kq | Config file: /etc/sagemaker-ts.properties
daikjnsgyj-algo-1-uf0kq | Inference address: http://0.0.0.0:8080
daikjnsgyj

In [24]:
predictions = predictor.predict(X.iloc[:,:].to_numpy().astype('float32')).squeeze()

daikjnsgyj-algo-1-uf0kq | 2022-05-31 23:19:47,872 [INFO ] W-9000-model_1 org.pytorch.serve.wlm.WorkerThread - Backend response time: 2180
daikjnsgyj-algo-1-uf0kq | 2022-05-31 23:19:47,873 [INFO ] W-9000-model_1 TS_METRICS - W-9000-model_1.ms:2582|#Level:Host|#hostname:1ac5a51a3933,timestamp:1654039187
daikjnsgyj-algo-1-uf0kq | 2022-05-31 23:19:47,873 [INFO ] W-9000-model_1 TS_METRICS - WorkerThreadTime.ms:95|#Level:Host|#hostname:1ac5a51a3933,timestamp:null
daikjnsgyj-algo-1-uf0kq | 2022-05-31 23:19:47,895 [INFO ] W-9001-model_1 org.pytorch.serve.wlm.WorkerThread - Backend response time: 2208
daikjnsgyj-algo-1-uf0kq | 2022-05-31 23:19:47,895 [INFO ] W-9001-model_1 TS_METRICS - W-9001-model_1.ms:2602|#Level:Host|#hostname:1ac5a51a3933,timestamp:1654039187
daikjnsgyj-algo-1-uf0kq | 2022-05-31 23:19:47,895 [INFO ] W-9001-model_1 TS_METRICS - WorkerThreadTime.ms:79|#Level:Host|#hostname:1ac5a51a3933,timestamp:null
daikjnsgyj-algo-1-uf0kq | 2022-05-31 23:19:47,902 [INFO ] W-9000-model_1-std

In [25]:
predictions

array([[ -2.54108453,   0.70870566,  -2.44132209],
       [  3.46189785,   0.78206122,  -6.78806257],
       [-19.62211418,  -1.13226449,   8.29794312],
       ...,
       [ -1.52053976,   0.8749938 ,  -1.62585104],
       [ -2.36265445,   0.68617344,  -2.65089345],
       [ -1.52053976,   0.8749938 ,  -1.62585104]])

In [26]:
predictions = predictions.argmax(axis=1)

In [27]:
print(classification_report(dataset['ALVO'], predictions))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       365
           1       1.00      1.00      1.00       423
           2       1.00      1.00      1.00       758

    accuracy                           1.00      1546
   macro avg       1.00      1.00      1.00      1546
weighted avg       1.00      1.00      1.00      1546



In [28]:
sagemaker_session.delete_endpoint(
    endpoint_name = predictor.endpoint_name
)

ClientError: An error occurred (ValidationException) when calling the DeleteEndpoint operation: Could not find endpoint "arn:aws:sagemaker:us-east-1:160946455806:endpoint/datascience-campeonatos-mlp-6".

daikjnsgyj-algo-1-uf0kq | 2022-05-31 23:20:46,329 [INFO ] pool-2-thread-1 TS_METRICS - CPUUtilization.Percent:0.0|#Level:Host|#hostname:1ac5a51a3933,timestamp:1654039246
daikjnsgyj-algo-1-uf0kq | 2022-05-31 23:20:46,329 [INFO ] pool-2-thread-1 TS_METRICS - DiskAvailable.Gigabytes:7.577537536621094|#Level:Host|#hostname:1ac5a51a3933,timestamp:1654039246
daikjnsgyj-algo-1-uf0kq | 2022-05-31 23:20:46,329 [INFO ] pool-2-thread-1 TS_METRICS - DiskUsage.Gigabytes:100.53646469116211|#Level:Host|#hostname:1ac5a51a3933,timestamp:1654039246
daikjnsgyj-algo-1-uf0kq | 2022-05-31 23:20:46,329 [INFO ] pool-2-thread-1 TS_METRICS - DiskUtilization.Percent:93.0|#Level:Host|#hostname:1ac5a51a3933,timestamp:1654039246
daikjnsgyj-algo-1-uf0kq | 2022-05-31 23:20:46,329 [INFO ] pool-2-thread-1 TS_METRICS - MemoryAvailable.Megabytes:2528.125|#Level:Host|#hostname:1ac5a51a3933,timestamp:1654039246
daikjnsgyj-algo-1-uf0kq | 2022-05-31 23:20:46,329 [INFO ] pool-2-thread-1 TS_METRICS - MemoryUsed.Megabytes:1166.